In [1]:
#import necessary packages/libraries
from __future__ import division
from numpy import *
import numpy as np
import fractions
import copy
import heapq
from fractions import Fraction

In [2]:
class Tableau:
    
    def __init__(self, obj):
        self.obj =  [0] + obj #store LP objective function
        self.rows = [] # store tableau rows as list of numpy arrays
        self.cons = [] #store b-vector of constraints; note in standard form, all b-vector constraints must be nonnegative
        self.feasible = True
        self.bounded = True
        self.objholder = [i for i, e in enumerate(self.obj) if e != 0] #for indexing sourcerows
        self.labels = [] #store x1, x2, x3,..., LP variable labels
        self.artifs = [] #store a1, a2, a3,...,an artificial variable labels
        self.wrow = [0] #store wrow values for artificial variables, [0] denotes no wrow since, no artificial vars added
        self.eqtypes = [] #encode 0 as <=, 1 as =, >= as 2.
        self.eqs = [] #store row positions of equality constraints
        self.geqs = [] #store row positions of greater than or equal constraints
        self.leqs = [] #store row positions of less than or equal constrains
        self.ineqs = [] #concatenation of leqs and geqs
        self.arows = [] #store positions of geqs and eqs for rows needing artificial variables
        self.step_counter = 0
        self.step_ab = ""

    def add_constraint(self, expression, value, eq = None):
        self.rows.append([0] + expression)
        self.cons.append(value)
        if eq == '<=':
            self.eqtypes.append(0)
        elif eq == '=':
            self.eqtypes.append(1)
        elif eq == '>=':
            self.eqtypes.append(2)
        elif eq == None:
            self.eqtypes.append(0) #default, if relation omitted, constraint assumed to be <=
        
    def _pivot_column(self):
        low = 0
        idx = 0
        for i in range(1, len(self.obj)-1):
            if self.obj[i] < low:
                low = self.obj[i]
                idx = i
        if idx == 0: return -1
        return idx
 
    def _pivot_row(self, col):
        rhs = [self.rows[i][-1] for i in range(len(self.rows))]
        lhs = [self.rows[i][col] for i in range(len(self.rows))]
        ratio = []
        for i in range(len(rhs)):
            if lhs[i] <= 0:
                ratio.append(float("inf"))
                continue
            ratio.append(rhs[i]/lhs[i])
            self.cons[i] = rhs[i]
        return argmin(ratio) if min(ratio) < float("inf") else 'u' 
    
    #PIVOT OPERATION used in both simplex & dual simplex
    def _pivot(self, row, col):
        e = self.rows[row][col]
        self.rows[row] /= e  
        for r in range(len(self.rows)):
            if r == row: continue
            self.rows[r] = self.rows[r] - self.rows[r][col]*self.rows[row]    
        self.obj = self.obj - self.obj[col]*self.rows[row]
        self.cons = [self.rows[i][-1] for i in range(len(self.rows))]
        
    def _wpivot(self, row, col):
        e = self.rows[row][col]
        self.rows[row] /= e  
        for r in range(len(self.rows)):
            if r == row: continue
            self.rows[r] = self.rows[r] - self.rows[r][col]*self.rows[row]    
        self.obj = self.obj - self.obj[col]*self.rows[row]
        self.wrow = self.wrow - self.wrow[col]*self.rows[row]
        self.cons = [self.rows[i][-1] for i in range(len(self.rows))]
 
    def _check(self):
        m = min(self.obj[1:-1])
        if Fraction(m).limit_denominator().numerator >= 0: return 1
        return 0
         
    def _wcheck(self):
        m = min(self.wrow[1:-(len(self.arows)+1)])
        if Fraction(m).limit_denominator().numerator >= 0: return 1
        return 0
    
    def _wpivot_column(self):
        low = 0
        idx = 0
        for i in range(1, len(self.wrow)-(len(self.arows)+1)):
            if self.wrow[i] < low:
                low = self.wrow[i]
                idx = i
        if idx == 0: return -1
        return idx
    
    def simplex_solve(self):
        if self.step_counter == 0:
            self.step_counter += 1
            print("Tableau {}".format(self.step_counter))
        else: 
            prev_tab = self.step_counter
            this_tab = prev_tab + 1
            self.step_counter += 1
            print("Tableau {} From Tableau {}".format(this_tab, prev_tab))
        
        if list(self.wrow) != [0]:
            #primal simplex solve on a phase I LP Tableau
            print('Simplex Phase I Initiated\n')
            self.display()
            while not self._wcheck():
                c = self._wpivot_column()
                r = self._pivot_row(c)
                if r == 'u':  ##code to deal with unbounded case
                    print('No Basic Feasible Solution')
                    self.bounded = True
                    self.feasible = False
                    break
                self._wpivot(r,c)
                print ('\npivot row: %s\npivot column: %s'%(r+1,c))
                self.display()
            print('Simplex Phase I Completed\n')
            
            if min(self.wrow[1:-(len(self.arows)+1)]) >= 0 and Fraction(self.wrow[-1]).limit_denominator().numerator != 0:
                print('No Basic Feasible Solution')
                #set feasible flag to false
                self.feasible = False
            else:
                #reset self.wrow = [0] and run phase II solve
                self.wrow = [0]
                #reconstruct the tableau without wrow and and artificials, i.e. remove artificials columns from each row
                for p in range(len(self.rows)):
                    self.rows[p] = np.append(self.rows[p][:-(len(self.arows)+1)], self.rows[p][-1])
                self.obj = np.append(self.obj[:-(len(self.arows)+1)], self.obj[-1])
                 
                print('Simplex Phase II Initiated\n')
                #solve the phase II problem
                self.display()
                while not self._check():
                    c = self._pivot_column()
                    r = self._pivot_row(c)
                    if r == 'u':  ##code to deal with unbounded case
                        print('Unbounded Condition Encountered')
                        self.bounded = False
                        break
                    self._pivot(r,c)
                    print ('\npivot row: %s\npivot column: %s'%(r+1,c))
                    self.display()            
                print('Simplex Phase II Completed\n')
        else:
            #primal simplex solve for LPs that only require phase II pivoting
            print('Simplex Phase II Initiated\n')
            self.display()
            while not self._check():
                c = self._pivot_column()
                r = self._pivot_row(c)
                if r == 'u':  ##code to deal with unbounded case
                    print('Unbounded Condition Encountered')
                    self.bounded = False
                    break
                self._pivot(r,c)
                print ('\npivot row: %s\npivot column: %s'%(r+1,c))
                self.display()            
            print('Simplex Phase II Completed\n')
                
    #For inserting new sensitivity constraints to relaxed solution tableau during each branch stage of BranchAndBound Solver
    def add_new_const(self, expression, value):
        pad = len(expression + [value])
        padding = pad - (max([len(self.rows[i]) for i in range(len(self.rows))]) -1)
        if padding > 0:
            for i in range(len(self.rows)):
                self.rows[i] = np.insert(self.rows[i], -1,np.zeros(padding))
            self.obj = np.insert(self.obj, -1,np.zeros(padding))
        self.rows.append(array([0] + expression + [value], dtype=float))
        self.cons.append(value)
        return self

    def _dualpivot_row(self):
        low = 0
        idx = -1
        for i in range(len(self.cons)):
            if self.cons[i] < low:
                low = self.cons[i]
                idx = i
        return idx
        
    def _dualcheck(self):
        m = min(self.cons)
        if Fraction(m).limit_denominator().numerator >= 0: return 1
        return 0
            
    def dualsimplex_solve(self, left=None, prev_ab=None):
        #dual solve
        if self.step_counter == 0:
            self.step_counter += 1
            print("Tableau {}".format(self.step_counter))
        else: 
            prev_tab = self.step_counter
            this_tab = prev_tab + 1
            ab = None if left == None else "a" if left == True else "b"
            self.step_counter += 1
            self.step_ab = ab
            
            print("Tableau {} From Tableau {}".format(str(this_tab) + ab, str(prev_tab) + prev_ab))
              
        self.display()
        while not self._dualcheck():
            r = self._dualpivot_row()
            if r == -1:
                print("Infeasible")
                self.bounded = False 
                self.feasible = False
                break
            c = self._dualpivot_column(r)
            #exception handle if no dual pivots are available
            #since such case will mean that (99999999*(1+abs(max(obs))))  is in the entire  ratio list
            #its min will be at least 99999999, so dualpivot_column returns 'u' token and we break 
            if c == 'u': 
                print("Infeasible")
                self.bounded = False 
                self.feasible = False
                break
            self._pivot(r, c)
            print ('\npivot row: %s\npivot column: %s'%(r+1,c))
            self.display()
        print('Dual Simplex Pivots Completed\n')
            
    def _dualpivot_column(self, row):
        obs = [self.obj[i] for i in range(len(self.obj[:-1]))] #objective function values for numerator
        ccs = [self.rows[row][i] for i in range(len(self.rows[row][:-1]))] #constraint side values for denominator
        ratio = []
        for i in range(len(obs)):
            if ccs[i] >= 0:
                ratio.append(float("inf"))
                continue
            ratio.append(abs(obs[i]/ccs[i]))
        
        self.cons = self.rows[:][-1]
        return np.argmin(ratio) if min(ratio) < float("inf") else 'u'

    def is_integer_solution(self):
        accumulator = 0.
        values = [i[2] for i in self.get_solution_vector() if i[1] in self.objholder]
        for v in values:
            accumulator += v;   
            
        f = Fraction(accumulator).limit_denominator()
        int_sol = (f.numerator / f.denominator) % 1 == 0.0
        return (f.numerator / f.denominator) % 1 == 0.0
    
    def get_solution_vector(self):
        sol_vec = []
        
        #For each column
        for i in self.objholder:
            x_index = None
            basis_variable = True
            #For each element of the column
            for j in range(len(self.rows)):
                e = self.rows[j][i]
                #Only 0 or 1 in basis
                if e != 0.0 and e != 1.0:
                    basis_variable = False
                    break
                #Found basis index?
                elif e == 1:
                    #Yes if x_index hasn't been set
                    if x_index == None:
                        x_index = j
                    #No if x_index has already been set
                    else:
                        basis_variable = False
                        break
            #Append if we found a basis variable
            if basis_variable == True and x_index != None:
                sol_vec.append((x_index, i, self.cons[x_index]))
        
        return sol_vec       
            
    
    def debugdisplay(self):
        if self.wrow is not [0]:
            np.set_printoptions(formatter={'float':lambda x: str(fractions.Fraction(x).limit_denominator())}) #remove for decimals
            P = np.asarray(matrix(self.rows + [self.obj] + [self.wrow])) #[:,1:]
            print('\n', P)
        else:
            np.set_printoptions(formatter={'float':lambda x: str(fractions.Fraction(x).limit_denominator())}) #remove for decimals
            print( '\n', matrix(self.rows + [self.obj])) #[:,1:]                       
    
    def display(self):
        if list(self.wrow) != [0]:
            self.labels = ['x'+str(i) for i in range(1,len(self.obj[:-(len(self.arows)+1)]))]
            self.artifs = ['a'+str(j+1) for j in range(len(self.arows))] + [' ']
            header = self.labels + self.artifs
            P = np.asarray(matrix(self.rows + [self.obj] + [self.wrow])[:,1:])
            print(*["{0:>7}".format(q) for q in header])
            for row in P:
                for item in row:
                    print("{0:>6}".format(str(Fraction(item).limit_denominator())), end = "  ") # fractional print
                print("")
        else:
            #normal phase II only display
            self.labels = ['x'+str(i) for i in range(1,len(self.obj[:-1]))] + [' ']
            header = self.labels                     
            M = np.asarray(matrix(self.rows + [self.obj])[:,1:])
            print(*["{0:>7}".format(i) for i in header])
            for row in M:
                for item in row:
                    print("{0:>6}".format(str(Fraction(item).limit_denominator())), end = "  ") # fractional print
                print("")

    def artificial_matrix(self):
        #append artificial variables
        for i in range(len(self.rows)):
            artificials = [0 for r in range(len(self.arows))] 
            if i not in self.arows:
                artificials = artificials
                self.rows[i] = np.insert(self.rows[i], -1,artificials)
            elif i in self.eqs or i in self.geqs:
                    artificials[self.arows.index(i)] = 1
                    self.rows[i] = np.insert(self.rows[i], -1,artificials)

        #build wrow from rows with artificial variables
        for j in range(len(self.rows)):
            if j in self.arows:
                self.wrow += self.rows[j]
        self.wrow = -1 * self.wrow
        #append artificial columns to obj row
        self.obj += [0.0 for i in range(len(self.arows))]                   

    # build full tableau
    def build_tableau(self):
            #get row position idx's of <=, >= and = constraints; use ineqs for slack matrix, arows for artificial matrix
            self.leqs = [l for l, u in enumerate(self.eqtypes) if u == 0]
            self.eqs = [q for q , p in enumerate(self.eqtypes) if p == 1]
            self.geqs = [g for g, s in enumerate(self.eqtypes) if s == 2]
            self.ineqs = self.leqs + self.geqs
            self.arows = self.eqs + self.geqs
        
            for i in range(len(self.rows)):
                #control slack, first (for r in range(len(self.leqs)))
                #define the no. of slack columns needed equal to the no. of inequality rows 
                #build slack matrix
                slack = [0 for r in range(len(self.ineqs))]
                if i not in (self.ineqs):
                    slack = slack
                    self.rows[i] += slack + [self.cons[i]]
                elif i in self.leqs:
                    slack[self.ineqs.index(i)] = 1
                    self.rows[i] += slack + [self.cons[i]]
                elif i in self.geqs:
                    slack[self.ineqs.index(i)] = -1
                    self.rows[i] += slack + [self.cons[i]]
                self.rows[i] = np.array(self.rows[i], dtype=float)
                
            for i in range(len(self.ineqs)):
                self.obj += [0]
            self.obj = list(np.array(self.obj + [0], dtype=float))
            #append a matrix of artificial variables to the tableau if there are any >= or = constraints
            if self.arows != []:
                self.artificial_matrix() 

    def __lt__(self, other):
        return -self.obj[-1] < -other.obj[-1]

In [3]:
def lessthanbranch(Tableau):
    solution_vector = Tableau.get_solution_vector()
    modulo = np.modf([i[2] for i in solution_vector])
    #since q returns the idx of largest fractional, it is also the sourcerow identifier
    q = np.argmax(modulo[0]) #fractions in modulo[0], ints in modulo[1]
    floor = modulo[1][q]
    ceiling = modulo[1][q] + 1
    lessthanbranch = np.zeros(len(Tableau.rows[solution_vector[q][0]])+1)
    
    #copy basic variable from sourcerow
    lessthanbranch[solution_vector[q][1]] = 1
    #assign floor to last element of lessthanbranch
    lessthanbranch[-1] = floor
    #add new slack variable for constraint
    lessthanbranch[-2] = 1
    print(solution_vector[q])
    print(lessthanbranch)
    #create operating row by inserting a zero
    oprow = np.insert(Tableau.rows[solution_vector[q][0]],-1, 0)
    lessthanconstraint = lessthanbranch - oprow
    np.set_printoptions(formatter={'float':lambda x: str(fractions.Fraction(x).limit_denominator())})
    print('Branch on x'+str(solution_vector[q][1]), '<=', floor)
    print('Less than constraint added:\n', lessthanconstraint[1:])
    print('\n')
    return lessthanconstraint[1:]
    
def greaterthanbranch(Tableau):    
    solution_vector = Tableau.get_solution_vector()
    modulo = np.modf([i[2] for i in solution_vector])
    q = np.argmax(modulo[0]) #fractions in array[0], ints in modulo[1]
    ceiling = modulo[1][q] + 1    
    greaterthanbranch = np.zeros(len(Tableau.rows[solution_vector[q][0]])+1)
    
    greaterthanbranch[solution_vector[q][1]] = -1
    greaterthanbranch[-1] = -ceiling
    greaterthanbranch[-2] = 1
    oprow = np.insert(Tableau.rows[solution_vector[q][0]],-1, 0)
    greaterthanconstraint = greaterthanbranch + oprow
    np.set_printoptions(formatter={'float':lambda x: str(fractions.Fraction(x).limit_denominator())})
    print('Branch on x'+str(solution_vector[q][1]), '>=', ceiling)
    print('Greater than constraint added:\n', greaterthanconstraint[1:])
    print('\n')
    return greaterthanconstraint[1:]

In [4]:
def stage(Tableau):
    t = Tableau    
    copyt = copy.deepcopy(Tableau)
    
    less = lessthanbranch(t)
    LesserT = t.add_new_const(less[:-1].tolist(), less[-1])
    LesserT.dualsimplex_solve(left=True, prev_ab=t.step_ab)
    SolvedLesser = LesserT
    
    greater = greaterthanbranch(copyt)
    GreaterT = copyt.add_new_const(greater[:-1].tolist(), greater[-1])
    GreaterT.dualsimplex_solve(left=False, prev_ab=copyt.step_ab)
    SolvedGreater = GreaterT
    
    #return a tuple of tableau's corresponding to the Lessthan and Greaterthan branches
    return (SolvedLesser, SolvedGreater)
    

In [5]:
class PriorityQueue:
    def __init__(self):
        self.heap = [];
    def push(self, obj):
        t = (-obj.obj[-1], obj)
        heapq.heappush(self.heap, t)
    def pop(self):
        return heapq.heappop(self.heap)
    def empty(self):
        return False if len(self.heap) > 0 else True

In [6]:
def BranchAndBound(tableau):
    tableau.simplex_solve()
    integer_solution = (float("inf"), tableau)
    pq = PriorityQueue()
    pq.push(tableau)
    
    while not pq.empty():        
        t = pq.pop()
        if t[1].is_integer_solution():
            if (t[0] < integer_solution[0]):
                integer_solution = t
                continue
        if (t[0] > integer_solution[0]):
            continue
        new_ts = stage(t[1])
        
        if (new_ts[0].feasible == True):
            pq.push(new_ts[0])
        if (new_ts[1].feasible == True):
            pq.push(new_ts[1])
        
    return integer_solution[1] 

In [7]:
t1 = Tableau([-3,-4])
t1.add_constraint([2, 1], 6)
t1.add_constraint([2, 3], 9)
t1.build_tableau()
t1.simplex_solve()
print(t1.is_integer_solution())

t2 = Tableau([-1,-2])
t2.add_constraint([3, 1], 7)
t2.add_constraint([2, 3], 11)
t2.build_tableau()
t2.simplex_solve()

t3 = Tableau([-2,-3,-3])
t3.add_constraint([3,2,0], 60)
t3.add_constraint([-1,1,4], 10)
t3.add_constraint([2,-2,5], 50)
t3.build_tableau()
t3.simplex_solve()
print(t3.is_integer_solution())

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
     2       1       1       0       6  
     2       3       0       1       9  
    -3      -4       0       0       0  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4        
   4/3       0       1    -1/3       3  
   2/3       1       0     1/3       3  
  -1/3       0       0     4/3      12  

pivot row: 1
pivot column: 1
     x1      x2      x3      x4        
     1       0     3/4    -1/4     9/4  
     0       1    -1/2     1/2     3/2  
     0       0     1/4     5/4    51/4  
Simplex Phase II Completed

False
Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
     3       1       1       0       7  
     2       3       0       1      11  
    -1      -2       0       0       0  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4        
   7/3       0       1    -1/3    10/3  
   2/3       1       0     1/3    11/3  
   1/3       0       0    

# SUCCESSFUL

In [8]:
#Success?
#ERROR: Phase I & II pass to BnB Solver Issue
#list index out of range
#Phase I solve completes with a non-integer solution, so there should be branching
#but I suspect there is a dimensions issue since phase I ends by cutting artificial columns from the initial tableau.

p3test = Tableau([1,2,0,-1])
p3test.add_constraint([1,-1,3,0], 1, '=')
p3test.add_constraint([0,1,-2,1], 1, '=')
p3test.add_constraint([3,1,1,4], 7, '=')
p3test.build_tableau()
p3test_sol = BranchAndBound(p3test)
#p3test.simplex_solve()
# print(p3test.cons)
# print(p3test.objholder)

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      a1      a2      a3        
     1      -1       3       0       1       0       0       1  
     0       1      -2       1       0       1       0       1  
     3       1       1       4       0       0       1       7  
     1       2       0      -1       0       0       0       0  
    -4      -1      -2      -5      -1      -1      -1      -9  

pivot row: 2
pivot column: 4
     x1      x2      x3      x4      a1      a2      a3        
     1      -1       3       0       1       0       0       1  
     0       1      -2       1       0       1       0       1  
     3      -3       9       0       0      -4       1       3  
     1       3      -2       0       0       1       0       1  
    -4       4     -12       0      -1       4      -1      -4  

pivot row: 1
pivot column: 3
     x1      x2      x3      x4      a1      a2      a3        
   1/3    -1/3       1       0     1/3       0       0     1/

In [9]:
#Success
#ERROR: Phase I & II pass to BnB Solver Issue
#list index out of range?
#bnb Solver throws error even though Phase I resulted in no BFS. There is a success case where it stopped after no BFS.
#this is a 3841HW LP problem that has no BFS by design. So there should be no branching at all.

p1test = Tableau([1,1,0,-1])
p1test.add_constraint([4,1,1,4], 8, '=')
p1test.add_constraint([1,-3,1,2], 16, '=')
p1test.build_tableau()
#p1test.simplex_solve()
p1test_solution = BranchAndBound(p1test)

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      a1      a2        
     4       1       1       4       1       0       8  
     1      -3       1       2       0       1      16  
     1       1       0      -1       0       0       0  
    -5       2      -2      -6      -1      -1     -24  

pivot row: 1
pivot column: 4
     x1      x2      x3      x4      a1      a2        
     1     1/4     1/4       1     1/4       0       2  
    -1    -7/2     1/2       0    -1/2       1      12  
     2     5/4     1/4       0     1/4       0       2  
     1     7/2    -1/2       0     1/2      -1     -12  

pivot row: 1
pivot column: 3
     x1      x2      x3      x4      a1      a2        
     4       1       1       4       1       0       8  
    -3      -4       0      -2      -1       1       8  
     1       1       0      -1       0       0       0  
     3       4       0       2       1      -1      -8  
Simplex Phase I Completed

No Basic Feasible Soluti

In [10]:
#Success?
#ERROR: Phase I & II pass to BnB Solver issue
#phase I & II problem example
#Simplex solve completes the initial phase I and II solve which already results in an integer solution,
#so we expect no branching, but an error is raised by the BnB solver

p2test = Tableau([1,1,1,3])
p2test.add_constraint([3,-1,3,6], 150, '=')
p2test.add_constraint([2, 2, -1, 4], 100, '=')
p2test.build_tableau()
#p2test.simplex_solve()
sol_p2test = BranchAndBound(p2test)

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      a1      a2        
     3      -1       3       6       1       0     150  
     2       2      -1       4       0       1     100  
     1       1       1       3       0       0       0  
    -5      -1      -2     -10      -1      -1    -250  

pivot row: 1
pivot column: 4
     x1      x2      x3      x4      a1      a2        
   1/2    -1/6     1/2       1     1/6       0      25  
     0     8/3      -3       0    -2/3       1       0  
  -1/2     3/2    -1/2       0    -1/2       0     -75  
     0    -8/3       3       0     2/3      -1       0  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4      a1      a2        
   1/2       0    5/16       1     1/8    1/16      25  
     0       1    -9/8       0    -1/4     3/8       0  
  -1/2       0   19/16       0    -1/8   -9/16     -75  
     0       0       0       0       0       0       0  
Simplex Phase I Completed

Simplex Phase II Initiat

In [11]:
#Success?
#ERROR: #Unorderable types error encountered in the priority queue
#Example from pg. 53 of slides from:
#https://ocw.mit.edu/courses/sloan-school-of-management/15-053-optimization-methods-in-management-science-spring-2013/lecture-notes/MIT15_053S13_lec12.pdf
#Relaxed has a solution, but there is no integer solution
#so every branch should terminate as infeasible

ex4 = Tableau([-24,-2,-20,-4])
ex4.add_constraint([8,1,5,4], 9)
ex4.add_constraint([1,0,0,0], 1, '<=')
ex4.add_constraint([0,1,0,0], 1, '<=')
ex4.add_constraint([0,0,1,0], 1, '<=')
ex4.add_constraint([0,0,0,1], 1, '<=')
ex4.build_tableau()
ex4_sol = BranchAndBound(ex4)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4      x5      x6      x7      x8      x9        
     8       1       5       4       1       0       0       0       0       9  
     1       0       0       0       0       1       0       0       0       1  
     0       1       0       0       0       0       1       0       0       1  
     0       0       1       0       0       0       0       1       0       1  
     0       0       0       1       0       0       0       0       1       1  
   -24      -2     -20      -4       0       0       0       0       0       0  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4      x5      x6      x7      x8      x9        
     0       1       5       4       1      -8       0       0       0       1  
     1       0       0       0       0       1       0       0       0       1  
     0       1       0       0       0       0       1       0       0       1  
     0       0       1       0       0     

In [12]:
#Success?
#ERROR!:
#Thie & Keough Example on pg. 243. The LP is given at the bottom of pg. 242 under Ex. 6.4.3
#The optimal z = 58 with x1 = 2 and x2 = 4.
#Our BnB Solver finds the correct optimal solution at Tableau 4a. But then it continues to branch without stopping.
tk3 = Tableau([-3,-13])
tk3.add_constraint([2,9], 40, '<=') # note default constraint relation is '<=', so the '<=' string can be omitted
tk3.add_constraint([11,-8], 82, '<=')
tk3.build_tableau()
sol_tk3 = BranchAndBound(tk3)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
     2       9       1       0      40  
    11      -8       0       1      82  
    -3     -13       0       0       0  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4        
   2/9       1     1/9       0    40/9  
 115/9       0     8/9       1  1058/9  
  -1/9       0    13/9       0   520/9  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4        
     0       1  11/115  -2/115    12/5  
     1       0   8/115   9/115    46/5  
     0       0  167/115   1/115   294/5  
Simplex Phase II Completed

(0, 2, 2.4000000000000004)
[0 0 1 0 0 1 2]
Branch on x2 <= 2.0
Less than constraint added:
 [0 0 -11/115 2/115 1 -2/5]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     0       1  11/115  -2/115       0    12/5  
     1       0   8/115   9/115       0    46/5  
     0       0  -11/115   2/115       1    -2/5  
     0       0  167/115   1/115       0  

In [13]:
#Success?
#Weird Bug
#This is a capital budgeting problem
#Example from pg.3 of http://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf
#correct value is z = 1000 with x1 = 1, x2 = 6
#solver does discover z = 1000 with x1 = 1 and x2 = 6 as a solution
#however branch4b is supposed to be infeasible according to the pdf link
# but the solver does a dual simplex pivot on "row 0"? which should be impossible since the print is (row +1)

ex1 = Tableau([-100, -150])
ex1.add_constraint([8000, 4000], 40000)
ex1.add_constraint([15, 30], 200)
ex1.build_tableau()
ex1_sol = BranchAndBound(ex1)


Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
  8000    4000       1       0   40000  
    15      30       0       1     200  
  -100    -150       0       0       0  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4        
  6000       0       1  -400/3  40000/3  
   1/2       1       0    1/30    20/3  
   -25       0       0       5    1000  

pivot row: 1
pivot column: 1
     x1      x2      x3      x4        
     1       0  1/6000   -1/45    20/9  
     0       1  -1/12000    2/45    50/9  
     0       0   1/240    40/9  9500/9  
Simplex Phase II Completed

(1, 2, 5.555555555555556)
[0 0 1 0 0 1 5]
Branch on x2 <= 5.0
Less than constraint added:
 [0 0 1/12000 -2/45 1 -5/9]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     1       0  1/6000   -1/45       0    20/9  
     0       1  -1/12000    2/45       0    50/9  
     0       0  1/12000   -2/45       1    -5/9  
     0       0   1/240    40/9       0

In [14]:
#Success?
#ERROR: #branches to infinity
#0-1 Knapsack Problem with a Mutual Exclusivity Constraint
#example from pg. 10 of http://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf
#correct answer is x1 = 1, x2 = 0, x3 = 1, x4 = 0 and z = 700
#Our BnB solver does discover integer solution of (1,0,1,0) and z = 700, but continues branching well past that.

ex2 = Tableau([-300,-90,-400, -150])
ex2.add_constraint([35000,10000,25000,90000],120000)
ex2.add_constraint([4,2,7,3], 12)
ex2.add_constraint([1,1,0,0], 1)
ex2.add_constraint([1,0,0,0], 1)
ex2.add_constraint([0,1,0,0], 1)
ex2.add_constraint([0,0,1,0], 1)
ex2.add_constraint([0,0,0,1], 1)
ex2.build_tableau()
ex2_sol = BranchAndBound(ex2)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4      x5      x6      x7      x8      x9     x10     x11        
 35000   10000   25000   90000       1       0       0       0       0       0       0  120000  
     4       2       7       3       0       1       0       0       0       0       0      12  
     1       1       0       0       0       0       1       0       0       0       0       1  
     1       0       0       0       0       0       0       1       0       0       0       1  
     0       1       0       0       0       0       0       0       1       0       0       1  
     0       0       1       0       0       0       0       0       0       1       0       1  
     0       0       0       1       0       0       0       0       0       0       1       1  
  -300     -90    -400    -150       0       0       0       0       0       0       0       0  

pivot row: 6
pivot column: 3
     x1      x2      x3      x4      x5      x6      x7     

     0       0       0       1       0       0       0       0       0       0       0      -1       1  
     1    4/29       0       0  7/145000   -5/29       0       0       0       0       0  111/29   -3/29  
     0   -4/29       0       0  -7/145000    5/29       0       1       0       0       0  -111/29   32/29  
     0       1       0       0       0       0       0       0       1       0       0       0       1  
     0    6/29       1       0  -1/36250    7/29       0       0       0       0       0  -51/29   39/29  
     0       0       0       0       0       0       0       0       0       0       1       1       0  
     0   -6/29       0       0  1/36250   -7/29       0       0       0       1       0   51/29  -10/29  
     0  990/29       0       0   1/290  1300/29       0       0       0       0       0  8550/29  19050/29  

pivot row: 8
pivot column: 2
     x1      x2      x3      x4      x5      x6      x7      x8      x9     x10     x11     x12        
     0       

In [15]:
#example from pg. 28 of slides here:
#https://ocw.mit.edu/courses/sloan-school-of-management/15-053-optimization-methods-in-management-science-spring-2013/lecture-notes/MIT15_053S13_lec12.pdf
#There is supposed to be no feasible solution to this LPs relaxed problem
#Solver correctly stops after Phase I finds no Basic Feasible Solution to Relaxed Problem
ex3 = Tableau([-24,-2,-20,-4])
ex3.add_constraint([8,1,5,4],9)
ex3.add_constraint([1,0,0,0], 1, '=')
ex3.add_constraint([0,1,0,0], 1, '=')
ex3.add_constraint([0,0,1,0], 1, '=')
ex3.add_constraint([0,0,0,1], 1, '<=')
ex3.build_tableau()
sol_ex3 = BranchAndBound(ex3)

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      x6      a1      a2      a3        
     8       1       5       4       1       0       0       0       0       9  
     1       0       0       0       0       0       1       0       0       1  
     0       1       0       0       0       0       0       1       0       1  
     0       0       1       0       0       0       0       0       1       1  
     0       0       0       1       0       1       0       0       0       1  
   -24      -2     -20      -4       0       0       0       0       0       0  
    -1      -1      -1       0       0       0      -1      -1      -1      -3  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4      x5      x6      a1      a2      a3        
     0       1       5       4       1       0      -8       0       0       1  
     1       0       0       0       0       0       1       0       0       1  
     0       1       0       0       0      

In [16]:
#Correctly solves Thie & Keough Example tree on pg. 241 despite using different x selection rule 
#optimal value is minimum z = -3, with x1 = 1 and x2 = 2. 
tk1 = Tableau([1,-2])
tk1.add_constraint([2, 1], 5)
tk1.add_constraint([-4,4], 5)
tk1.build_tableau()
sol_tk1 = BranchAndBound(tk1)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
     2       1       1       0       5  
    -4       4       0       1       5  
     1      -2       0       0       0  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4        
     3       0       1    -1/4    15/4  
    -1       1       0     1/4     5/4  
    -1       0       0     1/2     5/2  

pivot row: 1
pivot column: 1
     x1      x2      x3      x4        
     1       0     1/3   -1/12     5/4  
     0       1     1/3     1/6     5/2  
     0       0     1/3    5/12    15/4  
Simplex Phase II Completed

(1, 2, 2.5)
[0 0 1 0 0 1 2]
Branch on x2 <= 2.0
Less than constraint added:
 [0 0 -1/3 -1/6 1 -1/2]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     1       0     1/3   -1/12       0     5/4  
     0       1     1/3     1/6       0     5/2  
     0       0    -1/3    -1/6       1    -1/2  
     0       0     1/3    5/12       0    15/4  

pivot row: 3

In [17]:
#Solver correctly solves Thie & Keough pg.242 Example Branch And Bound Problem
#The optimal at z = 117, x1 = 9 and x2 = 3 is discovered at Tableau 4a; solver continues to branch until stage 6
tk2 = Tableau([-8,-15])
tk2.add_constraint([10,21],156, '<=')
tk2.add_constraint([2,1], 22, '<=')
tk2.build_tableau()
sol_tk2 = BranchAndBound(tk2)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
    10      21       1       0     156  
     2       1       0       1      22  
    -8     -15       0       0       0  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4        
 10/21       1    1/21       0    52/7  
 32/21       0   -1/21       1   102/7  
  -6/7       0     5/7       0   780/7  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4        
     0       1    1/16   -5/16    23/8  
     1       0   -1/32   21/32  153/16  
     0       0   11/16    9/16   957/8  
Simplex Phase II Completed

(0, 2, 2.875000000000001)
[0 0 1 0 0 1 2]
Branch on x2 <= 2.0
Less than constraint added:
 [0 0 -1/16 5/16 1 -7/8]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     0       1    1/16   -5/16       0    23/8  
     1       0   -1/32   21/32       0  153/16  
     0       0   -1/16    5/16       1    -7/8  
     0       0   11/16    9/16       0   957/8 

     0       0       0       0       0       0       0       8      15     116  
Dual Simplex Pivots Completed

Branch on x2 >= 5.0
Greater than constraint added:
 [0 0 1/21 0 0 0 0 -10/21 1 -19/21]


Tableau 6b From Tableau 5a
     x1      x2      x3      x4      x5      x6      x7      x8      x9        
     0       1    1/21       0       0       0       0  -10/21       0   86/21  
     1       0       0       0       0       0       0       1       0       7  
     0       0   -1/21       1       0       0       0  -32/21       0   82/21  
     0       0    1/21       0       1       0       0  -10/21       0   23/21  
     0       0       0       0       0       1       0      -1       0       2  
     0       0    1/21       0       0       0       1  -10/21       0    2/21  
     0       0    1/21       0       0       0       0  -10/21       1  -19/21  
     0       0     5/7       0       0       0       0     6/7       0   822/7  

pivot row: 7
pivot column: 8
     x1      x

In [18]:
#This is a Cutting Plane problem from Thie and Keough, pg. 236, problem 2a.
#Our BnB Solver successfully finds the optimal z = 13 with integer solutions at stage 3 and ceases branching.
tk4 = Tableau([-1,-3])
tk4.add_constraint([-1,3],6)
tk4.add_constraint([2,1],12)
tk4.build_tableau()
BranchAndBound(tk4)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
    -1       3       1       0       6  
     2       1       0       1      12  
    -1      -3       0       0       0  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4        
  -1/3       1     1/3       0       2  
   7/3       0    -1/3       1      10  
    -2       0       1       0       6  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4        
     0       1     2/7     1/7    24/7  
     1       0    -1/7     3/7    30/7  
     0       0     5/7     6/7   102/7  
Simplex Phase II Completed

(0, 2, 3.4285714285714284)
[0 0 1 0 0 1 3]
Branch on x2 <= 3.0
Less than constraint added:
 [0 0 -2/7 -1/7 1 -3/7]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     0       1     2/7     1/7       0    24/7  
     1       0    -1/7     3/7       0    30/7  
     0       0    -2/7    -1/7       1    -3/7  
     0       0     5/7     6/7       0   102/7 

In [19]:
#original base test case problem from class notes week 1
#Solver successfully finds optimal z = 12 with x1 = 0, x2 = 3 and ceases branching
a = Tableau([-3,-4])
a.add_constraint([2,1], 6)
a.add_constraint([2,3], 9)
a.build_tableau()
a_sol = BranchAndBound(a)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
     2       1       1       0       6  
     2       3       0       1       9  
    -3      -4       0       0       0  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4        
   4/3       0       1    -1/3       3  
   2/3       1       0     1/3       3  
  -1/3       0       0     4/3      12  

pivot row: 1
pivot column: 1
     x1      x2      x3      x4        
     1       0     3/4    -1/4     9/4  
     0       1    -1/2     1/2     3/2  
     0       0     1/4     5/4    51/4  
Simplex Phase II Completed

(1, 2, 1.5000000000000004)
[0 0 1 0 0 1 1]
Branch on x2 <= 1.0
Less than constraint added:
 [0 0 1/2 -1/2 1 -1/2]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     1       0     3/4    -1/4       0     9/4  
     0       1    -1/2     1/2       0     3/2  
     0       0     1/2    -1/2       1    -1/2  
     0       0     1/4     5/4       0    51/4  

In [20]:
#Midterm Example Problem
#Solver correctly discovers the optimal integral solution and ceases to branch
ttest = Tableau([-9,-25])
ttest.add_constraint([7, 23], 160)
ttest.add_constraint([3, -11], 44)
ttest.build_tableau()
sol = BranchAndBound(ttest)

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4        
     7      23       1       0     160  
     3     -11       0       1      44  
    -9     -25       0       0       0  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4        
  7/23       1    1/23       0  160/23  
146/23       0   11/23       1  2772/23  
-32/23       0   25/23       0  4000/23  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4        
     0       1   3/146  -7/146   86/73  
     1       0  11/146  23/146  1386/73  
     0       0   87/73   16/73  14624/73  
Simplex Phase II Completed

(1, 1, 18.986301369863014)
[0 1 0 0 0 1 18]
Branch on x1 <= 18.0
Less than constraint added:
 [0 0 -11/146 -23/146 1 -72/73]


Tableau 2a From Tableau 1
     x1      x2      x3      x4      x5        
     0       1   3/146  -7/146       0   86/73  
     1       0  11/146  23/146       0  1386/73  
     0       0  -11/146  -23/146       1  -72/73  
     0       0   87/73   16/7

In [21]:
ct = Tableau([1,4,6])
ct.add_constraint([-2,4,1],8, '>=')
ct.add_constraint([1,4,2],12, '>=')
ct.add_constraint([2,0,2], 16, '>=')
ct.build_tableau()

In [22]:
ct.simplex_solve()

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      x6      a1      a2      a3        
    -2       4       1      -1       0       0       1       0       0       8  
     1       4       2       0      -1       0       0       1       0      12  
     2       0       2       0       0      -1       0       0       1      16  
     1       4       6       0       0       0       0       0       0       0  
    -1      -8      -5       1       1       1      -1      -1      -1     -36  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4      x5      x6      a1      a2      a3        
  -1/2       1     1/4    -1/4       0       0     1/4       0       0       2  
     3       0       1       1      -1       0      -1       1       0       4  
     2       0       2       0       0      -1       0       0       1      16  
     3       0       5       1       0       0      -1       0       0      -8  
    -5       0      -3      -1       1      

In [23]:
ctgift = Tableau([0.1,.02,.02,.02,.02,.02,.02])
ctgift.add_constraint([1,1,0,0,0,0,0],4000,'>=')
ctgift.add_constraint([1,-.02,1,0,0,0,0],2000,'>=')
ctgift.add_constraint([1,1,-.02,1,0,0,0],5000,'>=')
ctgift.add_constraint([1,1,1,-.02,1,0,0],-1000,'>=')
ctgift.add_constraint([1,1,1,1,-.02,1,0],0,'>=')
ctgift.add_constraint([1,1,1,1,1,-.02,1],-3000,'>=')
ctgift.add_constraint([-.1,1,1,1,1,1,-.02],-15000,'>=')
ctgift.build_tableau()

In [24]:
ctgift.simplex_solve()

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      x6      x7      x8      x9     x10     x11     x12     x13     x14      a1      a2      a3      a4      a5      a6      a7        
     1       1       0       0       0       0       0      -1       0       0       0       0       0       0       1       0       0       0       0       0       0    4000  
     1   -1/50       1       0       0       0       0       0      -1       0       0       0       0       0       0       1       0       0       0       0       0    2000  
     1       1   -1/50       1       0       0       0       0       0      -1       0       0       0       0       0       0       1       0       0       0       0    5000  
     1       1       1   -1/50       1       0       0       0       0       0      -1       0       0       0       0       0       0       1       0       0       0   -1000  
     1       1       1       1   -1/50       1       0       0       0       0 

     0       0       0   51/50  -51/50       1       0       0       0       0       1      -1       0       0       0       0       0      -1       1       0       0    1000  
     0       0  -51/50  -25999/27500   -1/50  -51/55  51/2750       0       1       0  -499/550       0       0   51/55       0      -1       0  499/550       0       0  -51/55  121020/11  
     0       1       1  499/550       1   10/11   -1/55       0       0       0   -1/11       0       0  -10/11       0       0       0    1/11       0       0   10/11  -151000/11  
     0       0       0  2601/27500       0  51/550  51/2750       0       0       0  51/550       0       0   -4/55       0       0       0  -51/550       0       0    4/55  -10980/11  
     0       0  101/50  -103/50  151/50      -1       0       1       0       1      -3       1       0       0      -1       0      -1       3      -1       0       0  -12000  

pivot row: 5
pivot column: 11
     x1      x2      x3      x4      x5      x6      x7 

In [25]:
solve412 = Tableau([1,4,6])
solve412.add_constraint([-2,4,1],8,'>=')
solve412.add_constraint([1,4,2],12,'>=')
solve412.add_constraint([2,0,2],16,'>=')
solve412.build_tableau()

In [26]:
solve412.simplex_solve()

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      x6      a1      a2      a3        
    -2       4       1      -1       0       0       1       0       0       8  
     1       4       2       0      -1       0       0       1       0      12  
     2       0       2       0       0      -1       0       0       1      16  
     1       4       6       0       0       0       0       0       0       0  
    -1      -8      -5       1       1       1      -1      -1      -1     -36  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4      x5      x6      a1      a2      a3        
  -1/2       1     1/4    -1/4       0       0     1/4       0       0       2  
     3       0       1       1      -1       0      -1       1       0       4  
     2       0       2       0       0      -1       0       0       1      16  
     3       0       5       1       0       0      -1       0       0      -8  
    -5       0      -3      -1       1      

In [27]:
solve414 = Tableau([3,1,4,2])
solve414.add_constraint([1,1,-2,1], 10, '=')
solve414.add_constraint([4,1,2,3],20,'>=')
solve414.build_tableau()

In [28]:
solve414.simplex_solve()

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      a1      a2        
     1       1      -2       1       0       1       0      10  
     4       1       2       3      -1       0       1      20  
     3       1       4       2       0       0       0       0  
    -5      -2       0      -4       1      -1      -1     -30  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4      x5      a1      a2        
     0     3/4    -5/2     1/4     1/4       1    -1/4       5  
     1     1/4     1/2     3/4    -1/4       0     1/4       5  
     0     1/4     5/2    -1/4     3/4       0    -3/4     -15  
     0    -3/4     5/2    -1/4    -1/4      -1     1/4      -5  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4      x5      a1      a2        
     0       1   -10/3     1/3     1/3     4/3    -1/3    20/3  
     1       0     4/3     2/3    -1/3    -1/3     1/3    10/3  
     0       0    10/3    -1/3     2/3    -1/3    -2/3   -50/

In [29]:
#HW3
five8 = Tableau([37,29,33])
five8.add_constraint([0,-1,-1],-1)
five8.add_constraint([-1,-1,0],-1)
five8.add_constraint([-1,-1,-1],-1)
five8.add_constraint([-1,0,-1],-1)

In [30]:
five8.build_tableau()

In [31]:
five8.dualsimplex_solve()

Tableau 1
     x1      x2      x3      x4      x5      x6      x7        
     0      -1      -1       1       0       0       0      -1  
    -1      -1       0       0       1       0       0      -1  
    -1      -1      -1       0       0       1       0      -1  
    -1       0      -1       0       0       0       1      -1  
    37      29      33       0       0       0       0       0  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4      x5      x6      x7        
     0       1       1      -1       0       0       0       1  
    -1       0       1      -1       1       0       0       0  
    -1       0       0      -1       0       1       0       0  
    -1       0      -1       0       0       0       1      -1  
    37       0       4      29       0       0       0     -29  

pivot row: 4
pivot column: 3
     x1      x2      x3      x4      x5      x6      x7        
    -1       1       0      -1       0       0       1       0  
    -2       0       0 

In [32]:
five8dual = Tableau([-1,-1,-1,-1])
five8dual.add_constraint([0,1,1,1],37)
five8dual.add_constraint([1,1,1,0],29)
five8dual.add_constraint([1,0,1,1],33)
five8dual.build_tableau()

In [33]:
five8dual.simplex_solve()

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4      x5      x6      x7        
     0       1       1       1       1       0       0      37  
     1       1       1       0       0       1       0      29  
     1       0       1       1       0       0       1      33  
    -1      -1      -1      -1       0       0       0       0  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4      x5      x6      x7        
     0       1       1       1       1       0       0      37  
     1       1       1       0       0       1       0      29  
     0      -1       0       1       0      -1       1       4  
     0       0       0      -1       0       1       0      29  

pivot row: 3
pivot column: 4
     x1      x2      x3      x4      x5      x6      x7        
     0       2       1       0       1       1      -1      33  
     1       1       1       0       0       1       0      29  
     0      -1       0       1       0      -1       1      

In [34]:
five9 = Tableau([1,2])
five9.add_constraint([-2,-4],-4)
five9.add_constraint([-4,1],-2)
five9.add_constraint([2,-5],-5)
five9.add_constraint([-5,-1],-10)
five9.add_constraint([-3,-6],-9)
five9.build_tableau()

In [35]:
five9.dualsimplex_solve()

Tableau 1
     x1      x2      x3      x4      x5      x6      x7        
    -2      -4       1       0       0       0       0      -4  
    -4       1       0       1       0       0       0      -2  
     2      -5       0       0       1       0       0      -5  
    -5      -1       0       0       0       1       0     -10  
    -3      -6       0       0       0       0       1      -9  
     1       2       0       0       0       0       0       0  

pivot row: 4
pivot column: 1
     x1      x2      x3      x4      x5      x6      x7        
     0   -18/5       1       0       0    -2/5       0       0  
     0     9/5       0       1       0    -4/5       0       6  
     0   -27/5       0       0       1     2/5       0      -9  
     1     1/5       0       0       0    -1/5       0       2  
     0   -27/5       0       0       0    -3/5       1      -3  
     0     9/5       0       0       0     1/5       0      -2  

pivot row: 3
pivot column: 2
     x1      x2      x

In [36]:
five9dual = Tableau([-4,-2,-5,-10,-9])
five9dual.add_constraint([2,4,-2,5,3],1)
five9dual.add_constraint([4,-1,5,1,6],2)
five9dual.build_tableau()

In [37]:
five9dual.simplex_solve()

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4      x5      x6      x7        
     2       4      -2       5       3       1       0       1  
     4      -1       5       1       6       0       1       2  
    -4      -2      -5     -10      -9       0       0       0  

pivot row: 1
pivot column: 4
     x1      x2      x3      x4      x5      x6      x7        
   2/5     4/5    -2/5       1     3/5     1/5       0     1/5  
  18/5    -9/5    27/5       0    27/5    -1/5       1     9/5  
     0       6      -9       0      -3       2       0       2  

pivot row: 2
pivot column: 3
     x1      x2      x3      x4      x5      x6      x7        
   2/3     2/3       0       1       1    5/27    2/27     1/3  
   2/3    -1/3       1       0       1   -1/27    5/27     1/3  
     6       3       0       0       6     5/3     5/3       5  
Simplex Phase II Completed



In [38]:
two9 = Tableau([2,3])
two9.add_constraint([2,3],30)
two9.add_constraint([1,2],10,'>=')
two9.add_constraint([1,-1],0,'>=')
two9.build_tableau()

In [39]:
two9.simplex_solve()

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      a1      a2        
     2       3       1       0       0       0       0      30  
     1       2       0      -1       0       1       0      10  
     1      -1       0       0      -1       0       1       0  
     2       3       0       0       0       0       0       0  
    -2      -1       0       1       1      -1      -1     -10  

pivot row: 3
pivot column: 1
     x1      x2      x3      x4      x5      a1      a2        
     0       5       1       0       2       0      -2      30  
     0       3       0      -1       1       1      -1      10  
     1      -1       0       0      -1       0       1       0  
     0       5       0       0       2       0      -2       0  
     0      -3       0       1      -1      -1       1     -10  

pivot row: 2
pivot column: 2
     x1      x2      x3      x4      x5      a1      a2        
     0       0       1     5/3     1/3    -5/3    -1/3    40/

In [53]:
two11primal = Tableau([-6,-5,-4,-5,-6])
two11primal.add_constraint([1,1,1,1,1],3)
two11primal.add_constraint([5,4,3,2,1],14)
two11primal.build_tableau()

In [54]:
two11primal.simplex_solve()

Tableau 1
Simplex Phase II Initiated

     x1      x2      x3      x4      x5      x6      x7        
     1       1       1       1       1       1       0       3  
     5       4       3       2       1       0       1      14  
    -6      -5      -4      -5      -6       0       0       0  

pivot row: 2
pivot column: 1
     x1      x2      x3      x4      x5      x6      x7        
     0     1/5     2/5     3/5     4/5       1    -1/5     1/5  
     1     4/5     3/5     2/5     1/5       0     1/5    14/5  
     0    -1/5    -2/5   -13/5   -24/5       0     6/5    84/5  

pivot row: 1
pivot column: 5
     x1      x2      x3      x4      x5      x6      x7        
     0     1/4     1/2     3/4       1     5/4    -1/4     1/4  
     1     3/4     1/2     1/4       0    -1/4     1/4    11/4  
     0       1       2       1       0       6       0      18  
Simplex Phase II Completed



In [79]:
two11dual = Tableau([3,4])
two11dual.add_constraint([1,5],6,'>=')
two11dual.add_constraint([1,4],5,'>=')
two11dual.add_constraint([1,3],4,'>=')
two11dual.add_constraint([1,2],5,'>=')
two11dual.add_constraint([1,1],6,'>=')
two11dual.build_tableau()

In [80]:
two11dual.simplex_solve()

Tableau 1
Simplex Phase I Initiated

     x1      x2      x3      x4      x5      x6      x7      a1      a2      a3      a4      a5        
     1       5      -1       0       0       0       0       1       0       0       0       0       6  
     1       4       0      -1       0       0       0       0       1       0       0       0       5  
     1       3       0       0      -1       0       0       0       0       1       0       0       4  
     1       2       0       0       0      -1       0       0       0       0       1       0       5  
     1       1       0       0       0       0      -1       0       0       0       0       1       6  
     3       4       0       0       0       0       0       0       0       0       0       0       0  
    -5     -15       1       1       1       1       1      -1      -1      -1      -1      -1     -26  

pivot row: 1
pivot column: 2
     x1      x2      x3      x4      x5      x6      x7      a1      a2      a3      a4      a